# Online normalizer calculation for softmax in Mojo

This tutorial is about [online normalizer calculation for softmax](https://arxiv.org/abs/1805.02867). 
We use Mojo lang to implement two softmax algorithms: safe softmax and safe softmax with online normalizer calculation.


We will learn about some interesting aspects of the Mojo lang, such as how to use the move semantics in returning a large value from a function, which is more efficient, how to deepcopy a large value on the heap etc, and how to benchmark algorithms using the built-in `benchmark` package while avoiding some pitfalls.

## Extend the Matrix struct

below is the original matrix struct from [offical Mojo tutorial](https://github.com/modularml/mojo/blob/bf73717d79fbb79b4b2bf586b3a40072308b6184/examples/matmul.mojo#L37) 

In [2]:
alias type = DType.float32
from random import rand

struct Matrix[rows: Int, cols: Int]:
    var data: DTypePointer[type]

    # Initialize zeroeing all values
    fn __init__(inout self):
        self.data = DTypePointer[type].alloc(rows * cols)
        memset_zero(self.data, rows * cols)

    # Initialize taking a pointer, don't set any elements
    fn __init__(inout self, data: DTypePointer[type]):
        self.data = data

    ## Initialize with random values
    @staticmethod
    fn rand() -> Self:
        var data = DTypePointer[type].alloc(rows * cols)
        rand(data, rows * cols)
        return Self(data)

    fn __getitem__(self, y: Int, x: Int) -> Scalar[type]:
        return self.load[1](y, x)

    fn __setitem__(inout self, y: Int, x: Int, val: Scalar[type]):
        self.store[1](y, x, val)

    fn load[nelts: Int](self, y: Int, x: Int) -> SIMD[type, nelts]:
        return self.data.load[width=nelts](y * self.cols + x)

    fn store[nelts: Int](self, y: Int, x: Int, val: SIMD[type, nelts]):
        return self.data.store[width=nelts](y * self.cols + x, val)

There are a few inconvenient things:


First of all, user needs to [explicitly release the resources](https://github.com/modularml/mojo/blob/bf73717d79fbb79b4b2bf586b3a40072308b6184/examples/matmul.mojo#L201) allocated on the heap, e.g.
```
A.data.free()
B.data.free()
C.data.free()
```

Secondly, cannot initialize from another `Matrix`, i.e. lacking in `Copy constructor`



In [20]:
var m0 = Matrix[1,1].rand()
var m1 = m0;

error: Expression [20]:2:10: 'Matrix[1, 1]' is not copyable because it has no '__copyinit__'
var m1 = m0;
         ^~


expression failed to parse (no further compiler diagnostics)

Thirdly, cannot initialize using `move` semantics, i.e. lacking in `Move constructor`.

In [21]:
var m0 = Matrix[1,1].rand()
var m1 = m0^

error: Expression [21]:2:12: 'Matrix[1, 1]' is not copyable or movable because it has no '__copyinit__' or '__moveinit__' member
var m1 = m0^
           ^


expression failed to parse (no further compiler diagnostics)

Let's address these issues one by one.

First of all, let's be `RAII` by implementing the `__del__` method.
Note that the `self` parameter of the `__del__` method must be `owned`.

Let's also implement the `__copyinit__` and `__moveinit__` methods.

In [3]:
alias type = DType.float32
from random import rand


struct Matrix[rows: Int, cols: Int]:
    var data: DTypePointer[type]

    # Initialize zeroeing all values
    fn __init__(inout self):
        self.data = DTypePointer[type].alloc(rows * cols)
        memset_zero(self.data, rows * cols)

    # Initialize taking a pointer, don't set any elements
    fn __init__(inout self, data: DTypePointer[type]):
        self.data = data

    fn __copyinit__(inout self, existing: Self):
        var n = rows * cols
        self.data = DTypePointer[type].alloc(n)
        memcpy[rows * cols](self.data, existing.data)

    #fn __moveinit__(inout self, owned existing: Self):
        # self.data = existing.data
        # destructor of `existing` is disabled,
        # don't worry the resource is released twice

    fn __del__(owned self):
        self.data.free()

    ## Initialize with random values
    @staticmethod
    fn rand() -> Self:
        var data = DTypePointer[type].alloc(rows * cols)
        rand(data, rows * cols)
        return Self(data)

    fn __getitem__(self, y: Int, x: Int) -> Scalar[type]:
        return self.load[1](y, x)

    fn __setitem__(inout self, y: Int, x: Int, val: Scalar[type]):
        self.store[1](y, x, val)

    fn load[nelts: Int](self, y: Int, x: Int) -> SIMD[type, nelts]:
        return self.data.load[width=nelts](y * self.cols + x)

    fn store[nelts: Int](self, y: Int, x: Int, val: SIMD[type, nelts]):
        return self.data.store[width=nelts](y * self.cols + x, val)


In [23]:
var m0 = Matrix[1,1].rand()
var m1 = m0
print(m0[0, 0], "=", m1[0, 0])

0.67886471748352051 = 0.67886471748352051


also seem to found a bug with running mojo code in notebook...

execute the code below from command line is totally fine.

let's move on

In [24]:
var m2 = Matrix[1,1].rand()
var m3 = m2^

error: Expression [24] wrapper:16:7: 'm2' is uninitialized at the implicit return from this function
  def __mojo_repl_expr_body__() -> None:
      ^


Expression [24]:1:1: 'm2' declared here
var m2 = Matrix[1,1].rand()
^

expression failed to parse (no further compiler diagnostics)

## Implement the `Column` struct

### composition instead of inheritance

column is essentially a [n, 1] matrix, which means we can take advantage of the `Matrix` we had improved.

Mojo does not support struct extending struct,  so we cannot do something like 
```mojo
struct Column[nelts: Int](Matrix[nelts, 1])
```
We will use composition instead of inheritance.

### softmax algorithms shall be implemented as struct methods.

### implement `__eq__` to assist comparing two columns in unittest

In [11]:

import math
alias ESP = 0.000001
struct Column[nelts: Int]:
    alias Col = Matrix[nelts, 1]
    var elements: Self.Col
    fn __init__(inout self):
        self.elements.__init__()
    fn __init__(inout self, matrix: Matrix[nelts, 1]):
        self.elements = matrix
    @staticmethod
    fn rand() -> Self:
        return Self.Col.rand()
    fn __getitem__(self, i: Int) -> Scalar[type]:
        return self.elements[i, 0]
    fn __setitem__(inout self, i: Int, val: Scalar[type]):
        self.elements[i, 0] = val
    fn __copyinit__(inout self, existing: Self):
        self.elements = existing.elements
    fn softmax_unroll(self) -> Self:
        var x_max: Scalar[type]  = math.limit.neginf[type]()
        @unroll(20)
        for i in range(nelts):
            var x = self[i]
            if x > x_max:
                x_max = x
        var d: Scalar[type] = 0
        @unroll(20)
        for i in range(nelts):
            var x = self[i]
            d += math.exp(x - x_max)
        var probs = Self()
        @unroll(20)
        for i in range(nelts):
            var x = self[i]
            probs[i] = math.exp(x - x_max) / d
        return probs
    fn softmax(self) -> Self:
        var x_max: Scalar[type]  = math.limit.neginf[type]()
        for i in range(nelts):
            var x = self[i]
            if x > x_max:
                x_max = x
        var d: Scalar[type] = 0
        for i in range(nelts):
            var x = self[i]
            d += math.exp(x - x_max)
        var probs = Self()
        for i in range(nelts):
            var x = self[i]
            probs[i] = math.exp(x - x_max) / d
        return probs
    fn softmax_online(self) -> Self:
        var m = math.limit.neginf[type]()
        var d: Scalar[type] = 0
        for i in range(nelts):
            var x = self[i]
            var m_prev = m
            if x > m:
                m = x
            d = d * math.exp(m_prev - m) + math.exp(x - m)
        var probs = Self()
        for i in range(nelts):
            var x = self[i]
            probs[i] = math.exp(x - m) / d
        return probs
    fn __eq__(self, other: Self) -> Bool:
        for i in range(nelts):
            var diff = self[i] - other[i]
            if math.abs[type, 1](diff) > ESP:
                return False
        return True



## Unittest softmax calculation

In [12]:
var logits = Column[10].rand()
print("logits", logits[0], logits[1], logits[2], logits[3])
var probs = logits.softmax()
print("probabilities\t", probs[0], probs[1], probs[2], probs[3])
var probs_online = logits.softmax_online()
var probs_unrolled = logits.softmax_unroll()
var s: Scalar[type] = 0.0
for i in range(10):
    s += probs[i]
print("expected sum=1", "actual=", s)
print(probs == probs_online)
print(probs == probs_unrolled)

logits 0.98255026340484619 0.75335586071014404 0.072685882449150085 0.88470715284347534
probabilities	 0.15293651819229126 0.12161112576723099 0.061569001525640488 0.13868147134780884
expected sum=1 actual= 1.0
True
True


## Benchmark
tip: use `keep` to provides a hint to the compiler to not optimize the variable use away.
 

without `keep`:

In [17]:
from benchmark import Unit
from benchmark.compiler import keep
alias nelts = 10000000
fn bench[func: fn (Column[nelts]) -> None, name: StringLiteral]():
    var logits = Column[nelts].rand()
    @always_inline
    @parameter
    fn test_fn():
        func(logits)

    var report = benchmark.run[test_fn]()
    report.print(Unit.ms)
fn naive(logits: Column[nelts]):
    var x = logits.softmax()
fn naive_unrolled(logits: Column[nelts]):
    var x = logits.softmax_unroll()
fn online(logits: Column[nelts]):
    var x = logits.softmax_online()
print("naive")
bench[naive, "naive"]()
print("naive unrolled")
bench[naive_unrolled, "naive unrolled"]()
print("online normalization")
bench[online, "online normalization"]()

naive
---------------------
Benchmark Report (ms)
---------------------
Mean: 1.7e-14
Total: 1.7e-05
Iters: 1000000000
Warmup Mean: 1.6500000000000001e-05
Warmup Total: 3.3000000000000003e-05
Warmup Iters: 2
Fastest Mean: 1.7e-14
Slowest Mean: 1.7e-14

naive unrolled
---------------------
Benchmark Report (ms)
---------------------
Mean: 94.32451648
Total: 2358.1129120000001
Iters: 25
Warmup Mean: 94.027575999999996
Warmup Total: 188.05515199999999
Warmup Iters: 2
Fastest Mean: 94.32451648
Slowest Mean: 94.32451648

online normalization
---------------------
Benchmark Report (ms)
---------------------
Mean: 1.7e-14
Total: 1.7e-05
Iters: 1000000000
Warmup Mean: 1.8499999999999999e-05
Warmup Total: 3.6999999999999998e-05
Warmup Iters: 2
Fastest Mean: 1.7e-14
Slowest Mean: 1.7e-14



with `keep`:

In [ ]:
from benchmark import Unit
from benchmark.compiler import keep
alias nelts = 10000000
fn bench[func: fn (Column[nelts]) -> None, name: StringLiteral]():
    var logits = Column[nelts].rand()
    @always_inline
    @parameter
    fn test_fn():
        var probs = func(logits)
        keep(probs)

    var report = benchmark.run[test_fn]()
    report.print(Unit.ms)

In [ ]:
fn naive(logits: Column[nelts]):
    var probs = logits.softmax()
    keep(probs)
fn naive_unrolled(logits: Column[nelts]):
    var probs = logits.softmax_unroll()
    keep(probs)
fn online(logits: Column[nelts]):
    var probs = logits.softmax_online()
    keep(probs)
print("naive")
bench[naive, "naive"]()
print("naive unrolled")
bench[naive_unrolled, "naive unrolled"]()
print("online normalization")
bench[online, "online normalization"]()

naive
---------------------
Benchmark Report (ms)
---------------------
Mean: 101.07316920833334
Total: 2425.756061
Iters: 24
Warmup Mean: 97.554480999999996
Warmup Total: 195.10896199999999
Warmup Iters: 2
Fastest Mean: 101.07316920833333
Slowest Mean: 101.07316920833333

naive unrolled
---------------------
Benchmark Report (ms)
---------------------
Mean: 95.434057639999992
Total: 2385.8514409999998
Iters: 25
Warmup Mean: 92.094906499999993
Warmup Total: 184.18981299999999
Warmup Iters: 2
Fastest Mean: 95.434057640000006
Slowest Mean: 95.434057640000006

online normalization
---------------------
Benchmark Report (ms)
---------------------
Mean: 124.00298378947369
Total: 2356.0566920000001
Iters: 19
Warmup Mean: 124.7894295
Warmup Total: 249.57885899999999
Warmup Iters: 2
Fastest Mean: 1.7976931348623157e+308
Slowest Mean: 124.00298378947369



## interesting benchmark results
online normalization is worse than naive algorithm on CPU.

Hypothesis: on CPU, the calculation is not memory bounded, but computation bounded.

* How to prove/disprove our hypothesis?

Future work, Mojo will support compiling to GPU, it would be interseting to do the same benchmark in Mojo on GPU. 